In [1]:
import pandas as pd
import numpy as np


In [4]:

pd.options.display.max_columns = False


In [3]:
from datetime import datetime, timedelta

In [ ]:
import sys
sys.path.append('C:/Users/juan.martinez/OneDrive - WOM Colombia/Proyectos/Utils')

In [2]:
import consultas as cst

In [5]:
%%time
query = '''
WITH 
    BASE AS(
        SELECT /*+PARALLEL(2)*/ 
            A.SUBSCRIBER_ID,
            A.PERIODO_PROCESO_CODIGO,
            A.FECHA_BAJA,
            ROUND(A.FECHA_BAJA - A.FECHA_ALTA) AS ANTIGUEDAD,
            COALESCE(B.PREDICCION, 0) AS PREDICCION
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
        LEFT JOIN (
            SELECT  /*+PARALLEL(2)*/ 
                *
            FROM (
                SELECT  /*+PARALLEL(2)*/
                    SUBSCRIBER_ID,
                    PREDICCION,
                    ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN
                FROM AGG_DL_PJ_QUALITY
            )
            WHERE RN = 1
        )B ON A.SUBSCRIBER_ID = B.SUBSCRIBER_ID
        WHERE A.SERVICIO = 'Prepaid'
            AND A.MOVIMIENTO_TIPO_NOMBRE = 'CHURN MIGRACION'    
            AND A.PERIODO_PROCESO_CODIGO >= 202210
    )

SELECT  /*+PARALLEL(2)*/
    A.SUBSCRIBER_ID,
    A.PERIODO_PROCESO_CODIGO,
    A.FECHA_BAJA,
    A.ANTIGUEDAD,
    A.PREDICCION,
    C.GRUPO_SEG_RFM_MES_ACT,
    SUM(CASE
        WHEN TO_DATE(TO_CHAR(B.TIEMPO_RECARGA_DK), 'YYYY/MM/DD') BETWEEN A.FECHA_BAJA - 30 AND A.FECHA_BAJA THEN VALOR_CARGA
        ELSE 0
    END) RECARGAS_30,
    SUM(CASE
        WHEN TO_DATE(TO_CHAR(B.TIEMPO_RECARGA_DK), 'YYYY/MM/DD') BETWEEN A.FECHA_BAJA - 60 AND A.FECHA_BAJA - 30 THEN VALOR_CARGA
        ELSE 0
    END) RECARGAS_60,
    SUM(CASE
        WHEN TO_DATE(TO_CHAR(B.TIEMPO_RECARGA_DK), 'YYYY/MM/DD') BETWEEN A.FECHA_BAJA - 90 AND A.FECHA_BAJA - 60 THEN VALOR_CARGA
        ELSE 0
    END) RECARGAS_90
FROM BASE A
LEFT JOIN DWH_BODEGA_WOM.FCT_RECARGAS B ON A.SUBSCRIBER_ID = B.SUBSCRIBER_ID
LEFT JOIN MVALIENTE.TBL_SEG_RFM_PREPAGO_NEW C ON A.SUBSCRIBER_ID = C.SUBSCRIBER_ID AND A.PERIODO_PROCESO_CODIGO = C.PERIODO
GROUP BY
    A.SUBSCRIBER_ID,
    A.PERIODO_PROCESO_CODIGO,
    A.FECHA_BAJA,
    A.ANTIGUEDAD,
    A.PREDICCION,
    C.GRUPO_SEG_RFM_MES_ACT
'''

migraciones_pre = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 2 s
Wall time: 7min 8s


In [6]:
migraciones_pre.groupby('PERIODO_PROCESO_CODIGO')[['SUBSCRIBER_ID']].count()

,SUBSCRIBER_ID
PERIODO_PROCESO_CODIGO,
202210,15115
202211,16206
202212,17564
202301,14307
202302,11766
202303,14155
202304,9973


In [7]:
%%time
query = '''
SELECT /*+PARALLEL(2)*/
    A.SUBSCRIBER_ID,
    A.PERIODO_PROCESO_CODIGO,
    B.PLAN_NOMBRE,
    B.PLAN_REVENUE,
    C.BILL_STATE,
    C.FECHA_PAGO,
    D.CONSULTOR_CANAL_NOMBRE,
    D.CONSULTOR_REGIONAL_NOMBRE,
    D.CONSULTOR_CIUDAD_NOMBRE,
    D.CONSULTOR_ALMACEN_NOMBRE,
    D.CONSULTOR_NOMBRE_NIVEL_8,
    E.ALTA_PROB
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES B ON A.PLAN_COMERCIAL_DK = B.PLAN_COMERCIAL_DK
LEFT JOIN (
    SELECT /*+PARALLEL(2)*/ 
        *
    FROM (
        SELECT /*+PARALLEL(2)*/
            SUBS_ID,
            BILL_STATE,
            FECHA_PAGO,
            ROW_NUMBER() OVER (PARTITION BY SUBS_ID ORDER BY FECHA_PAGO DESC) AS RN 
        FROM CR_DL_PAGOS
        --WHERE SALDO = 0
    )
WHERE RN = 1
)C ON A.SUBSCRIBER_ID = C.SUBS_ID
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES D ON A.CONSULTOR_DK = D.CONSULTOR_DK
LEFT JOIN MODELO_GROSS_CHURN_V2 E ON A.SUBSCRIBER_ID = E.SUBSCRIBER_ID AND A.PERIODO_PROCESO_CODIGO = E.PERIODO_PROCESO_CODIGO
WHERE A.SERVICIO = 'Postpaid'
    AND A.MOVIMIENTO_TIPO_NOMBRE = 'GROSS ADDS MIGRACION'
    AND A.PERIODO_PROCESO_CODIGO >= 202210
'''

migraciones_pos = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 1.59 s
Wall time: 4min 24s


In [8]:
migraciones_pos.groupby('PERIODO_PROCESO_CODIGO')[['SUBSCRIBER_ID']].count()

,SUBSCRIBER_ID
PERIODO_PROCESO_CODIGO,
202210,15116
202211,16206
202212,17564
202301,14307
202302,11766
202303,14155
202304,9973


In [9]:
migraciones = pd.merge(
    migraciones_pre, 
    migraciones_pos, 
    how='left', 
    left_on=['SUBSCRIBER_ID', 'PERIODO_PROCESO_CODIGO'],
    right_on=['SUBSCRIBER_ID', 'PERIODO_PROCESO_CODIGO']
)

In [10]:
migraciones.to_csv('Migraciones_Pre_Pos.csv', index=False, sep='|')

In [11]:
migraciones.to_excel('Migraciones_Pre_Pos.xlsx', index=False)

In [15]:
migraciones = pd.read_csv('Migraciones_Pre_Pos.csv',sep=',')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 22086, saw 2


In [13]:
migraciones['RECARGA_PROM'] = round((migraciones['RECARGAS_30'] + migraciones['RECARGAS_60'] + migraciones['RECARGAS_90'])/3)

In [14]:
migraciones['RANGO_REC'].fillna(0, inplace=True)
migraciones['RANGO_REC'] = np.where(migraciones['RECARGA_PROM'] < 5000, 
                                    '1. 0-5K', 
                                    np.where(migraciones['RECARGA_PROM'] < 10000, 
                                             '2. 5k-10k', 
                                             np.where(migraciones['RECARGA_PROM'] < 15000, 
                                                      '3. 10k-15k', 
                                                      np.where(migraciones['RECARGA_PROM'] < 20000, 
                                                               '4. 15k-20k', 
                                                               '5. >20k'))))

migraciones['RANGO_REC'].value_counts()

KeyError: 'RANGO_REC'

In [ ]:
#PORTACION ABRIL POR CALIDAD DE GROSS
query=f'''
SELECT * 
FROM (
SELECT A.SUBSCRIBER_ID,
       A.PREDICCION,
       A.METODO_CAL,
       ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN  
FROM AGG_DL_PJ_QUALITY A
WHERE A.SUBSCRIBER_ID IN (
SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'))
WHERE RN=1
'''

In [ ]:
#usuarios que migraron OCTUBRE
query=f'''
SELECT A.SUBSCRIBER_ID,
       ROUND(A.FECHA_BAJA-A.FECHA_ALTA,0) AS ANTIGUEDAD
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
'''

In [7]:
#QUERY QUE JUNTA LA ANTIGUEDAD CON SEGRFM
query=f'''
WITH BASE AS(
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202303
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND A.PERIODO=202303) ,
BASE2 AS (SELECT A.SUBSCRIBER_ID,
       ROUND(A.FECHA_BAJA-A.FECHA_ALTA,0) AS ANTIGUEDAD
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202303
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
)
SELECT A.*,B.ANTIGUEDAD 
FROM BASE A
JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID 
'''

migraciones_pos = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
migraciones_pos.to_csv('Migraciones_Pos_MAR.csv', index=False, sep=',')

In [ ]:
query=f'''
SELECT SEG_RFM_MES_ANT,ROUND(AVG(ANTIGUEDAD),0) FROM (
WITH BASE AS(
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND A.PERIODO=202304) ,
BASE2 AS (SELECT A.SUBSCRIBER_ID,
       ROUND(A.FECHA_BAJA-A.FECHA_ALTA,0) AS ANTIGUEDAD
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
)
SELECT A.*,B.ANTIGUEDAD 
FROM BASE A
JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
) GROUP BY SEG_RFM_MES_ANT
ORDER BY SEG_RFM_MES_ANT ASC
'''

In [ ]:
#dia baja 
query=f'''
SELECT DIA_BAJA,PREDICCION,COUNT(*) AS TOTAL FROM (
SELECT  A.SUBSCRIBER_ID,
        TO_NUMBER(TO_CHAR(A.FECHA_BAJA,'DD')) AS DIA_BAJA,
        COALESCE(B.PREDICCION,0) AS PREDICCION ,
        ROW_NUMBER() OVER (PARTITION BY A.SUBSCRIBER_ID ORDER BY B.FECHA_PRED DESC) AS RN
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN AGG_DL_PJ_QUALITY B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN INACTIVIDAD' )
WHERE RN=1
GROUP BY DIA_BAJA,PREDICCION
ORDER BY DIA_BAJA ,PREDICCION
'''

In [17]:
#base
query=f'''
SELECT A.SUBSCRIBER_ID,
       A.PERIODO_PROCESO_CODIGO,
       A.FECHA_ALTA,
       ROUND(A.FECHA_BAJA-A.FECHA_ALTA,0) AS ANTIGUEDAD,
       COALESCE(B.PREDICCION,0) AS MODELO_CALIDAD  
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN AGG_DL_PJ_QUALITY B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
'''
base = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
#RECARGAS ULTIMO MES
query=f'''
SELECT SUBSCRIBER_ID,SUM(VALOR_CARGA)AS ULTIMO_MES,COUNT(*)AS TOTAL_RECARGAS_MES,
ROUND(SUM(VALOR_CARGA)/COUNT(*),0) AS PROMEDIO_RECARGA
FROM (
WITH BASE AS(
SELECT * 
FROM DWH_BODEGA_WOM.FCT_RECARGAS A
WHERE A.SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND   A.SERVICIO='Prepaid') ,
BASE2 AS (SELECT A.SUBSCRIBER_ID,
A.FECHA_BAJA
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
SELECT A.SUBSCRIBER_ID,A.VALOR_CARGA
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.FECHA_RECARGA BETWEEN B.FECHA_BAJA-30 AND B.FECHA_BAJA
)
GROUP BY SUBSCRIBER_ID
'''
recargas_mes = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
#PAQUETES ULTIMO MES
query=f'''
SELECT SUBSCRIBER_ID,SUM(VALOR)AS ULTIMO_MES_PAQ,COUNT(*)AS TOTAL_PAQUETES_MES,
ROUND(SUM(VALOR)/COUNT(*),0) AS PROMEDIO_PAQ
FROM (
WITH BASE AS(
SELECT * 
FROM DWH_BODEGA_WOM.FCT_PAQUETES A
WHERE A.SUBSCRIBER_ID IN (SELECT A.SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND   A.SERVICIO='Prepaid') ,
BASE2 AS (SELECT A.SUBSCRIBER_ID,
A.FECHA_BAJA
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202304
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
SELECT A.SUBSCRIBER_ID,A.VALOR
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.FECHA_RECARGA BETWEEN B.FECHA_BAJA-30 AND B.FECHA_BAJA
)
GROUP BY SUBSCRIBER_ID
'''
paquetes_mes = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [24]:
df_insumos=base.merge(recargas_mes,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(paquetes_mes,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.fillna(0)

In [27]:
df_insumos

,SUBSCRIBER_ID,PERIODO_PROCESO_CODIGO,FECHA_ALTA,ANTIGUEDAD,MODELO_CALIDAD,ULTIMO_MES,TOTAL_RECARGAS_MES,PROMEDIO_RECARGA,ULTIMO_MES_PAQ,TOTAL_PAQUETES_MES,PROMEDIO_PAQ
0,1113227444,202304,2022-12-30 16:40:23,108,0,18000.0,3.0,6000.0,18000.0,3.0,6000.0
1,1129580352,202304,2022-12-01 15:52:49,147,0,9000.0,2.0,4500.0,0.0,0.0,0.0
2,1115944833,202304,2022-12-05 15:16:28,127,0,3500.0,1.0,3500.0,0.0,0.0,0.0
3,1115376850,202304,2022-12-07 12:30:31,142,0,0.0,0.0,0.0,0.0,0.0,0.0
4,1113147710,202304,2022-12-16 12:48:12,125,0,27000.0,2.0,13500.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
14098,1328433571,202304,2023-03-26 08:12:27,20,0,11000.0,2.0,5500.0,5500.0,2.0,2750.0
14099,1129125652,202304,2021-09-05 11:27:47,599,0,0.0,0.0,0.0,0.0,0.0,0.0
14100,1115640664,202304,2021-09-09 16:17:26,598,0,27500.0,5.0,5500.0,27500.0,5.0,5500.0
14101,1153117263,202304,2021-08-20 08:48:21,606,0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_insumos.describe()

,SUBSCRIBER_ID,PERIODO_PROCESO_CODIGO,ANTIGUEDAD,MODELO_CALIDAD,ULTIMO_MES,TOTAL_RECARGAS_MES,PROMEDIO_RECARGA,ULTIMO_MES_PAQ,TOTAL_PAQUETES_MES,PROMEDIO_PAQ
count,1.410300e+04,14103.0,14103.000000,14103.000000,14103.000000,14103.000000,14103.000000,14103.000000,14103.000000,14103.000000
mean,1.290910e+09,202304.0,254.545487,0.160888,11542.562504,1.845636,5314.114727,5248.509679,0.861093,3194.231724
std,2.105541e+08,0.0,194.654319,0.367440,11897.419091,1.864107,5178.734140,8217.276857,1.263444,4671.732420
min,1.925831e+08,202304.0,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.185802e+09,202304.0,102.000000,0.000000,1000.000000,1.000000,1000.000000,0.000000,0.000000,0.000000
50%,1.265758e+09,202304.0,194.000000,0.000000,9500.000000,1.000000,5100.000000,0.000000,0.000000,0.000000
75%,1.319763e+09,202304.0,381.000000,0.000000,19500.000000,3.000000,6561.500000,7500.000000,1.000000,5500.000000
max,2.402342e+09,202304.0,749.000000,1.000000,231000.000000,29.000000,63000.000000,189000.000000,14.000000,63000.000000


In [28]:
query=f'''
WITH 
    BASE AS(
        SELECT 
            A.SUBSCRIBER_ID,
            A.PERIODO_PROCESO_CODIGO,
            A.MOVIMIENTO_NOMBRE,
            CASE
                WHEN A.MOVIMIENTO_NOMBRE = 'DESCONEXION' THEN 1
                ELSE 0
            END CHURN,
            CASE
                WHEN A.MOVIMIENTO_NOMBRE = 'DESCONEXION' THEN A.MOVIMIENTO_TIPO_NOMBRE
                ELSE '<--->'
            END TIPO_CHURN,
            A.FECHA_ALTA,
            A.FECHA_BAJA,
            ROUND(CASE
                WHEN A.FECHA_BAJA IS NOT NULL THEN A.FECHA_BAJA - A.FECHA_ALTA
                ELSE TO_DATE(TO_CHAR(A.PERIODO_PROCESO_CODIGO) || '30', 'YYYYMMDD') - A.FECHA_ALTA
            END) ANTIGUEDAD,
            COALESCE(B.PREDICCION, 0) AS PREDICCION
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
        LEFT JOIN (
            SELECT *
            FROM (
                SELECT 
                    SUBSCRIBER_ID,
                    PREDICCION,
                    ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN
                FROM AGG_DL_PJ_QUALITY
            )
            WHERE RN = 1
        )B ON A.SUBSCRIBER_ID = B.SUBSCRIBER_ID
        WHERE A.SERVICIO = 'Prepaid'
            AND A.PERIODO_PROCESO_CODIGO IN (202304)                
    )
    
SELECT 
    A.SUBSCRIBER_ID, 
    A.PERIODO_PROCESO_CODIGO, 
    A.MOVIMIENTO_NOMBRE, 
    A.CHURN, 
    A.TIPO_CHURN, 
    A.FECHA_ALTA, 
    A.FECHA_BAJA, 
    A.ANTIGUEDAD, 
    A.PREDICCION,
    SUM(CASE
        WHEN FECHA_RECARGA BETWEEN A.FECHA_BAJA - 30 AND A.FECHA_BAJA THEN VALOR_CARGA
        ELSE 0
    END) AS VALOR_CARGA,
    SUM(CASE
        WHEN FECHA_RECARGA BETWEEN A.FECHA_BAJA - 30 AND A.FECHA_BAJA THEN 1
        ELSE 0
    END) AS CANT_CARGA
FROM BASE A
LEFT JOIN DWH_BODEGA_WOM.FCT_RECARGAS B ON A.SUBSCRIBER_ID = B.SUBSCRIBER_ID
GROUP BY 
    A.SUBSCRIBER_ID, 
    A.PERIODO_PROCESO_CODIGO, 
    A.MOVIMIENTO_NOMBRE, 
    A.CHURN, 
    A.TIPO_CHURN, 
    A.FECHA_ALTA, 
    A.FECHA_BAJA, 
    A.ANTIGUEDAD, 
    A.PREDICCION
'''
ultimo_mes = cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: DPI-1080: connection was closed by ORA-3113

In [ ]:
#PAGOS DE LOS QUE MGIRARON 

query=f'''
SELECT *
FROM CR_DL_PAGOS A
WHERE A.SUBS_ID IN (
SELECT SUBSCRIBER_ID FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
'''

In [5]:
query=F'''
SELECT A.SUBS_ID,
       -1*SUM(A.PAGO) AS PAGOS,
       COUNT(*) AS FACTURAS_PAGADAS
FROM CR_DL_PAGOS A
WHERE A.SUBS_ID IN (
SELECT SUBSCRIBER_ID FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND SALDO=0
GROUP BY SUBS_ID
'''
pagadas= cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
pagadas

,SUBS_ID,PAGOS,FACTURAS_PAGADAS
0,192571857,178500,3
1,1112632030,198000,6
2,1112632056,150000,6
3,1112633059,66000,2
4,1112634035,198000,6
...,...,...,...
12891,2298674855,150000,6
12892,2302230088,125000,5
12893,2303612507,33829,1
12894,2305752723,217250,6


In [17]:
pagadas.describe()

,SUBS_ID,PAGOS,FACTURAS_PAGADAS
count,1.289600e+04,1.289600e+04,12896.000000
mean,1.239878e+09,1.526580e+05,5.067308
std,2.100932e+08,9.298343e+04,2.783817
min,1.925719e+08,1.650000e+04,1.000000
25%,1.152839e+09,8.850000e+04,3.000000
50%,1.197879e+09,1.500000e+05,6.000000
75%,1.251413e+09,1.980000e+05,6.000000
max,2.305967e+09,1.456510e+06,21.000000


In [14]:
pagadas['PAGOS'].sum()

1968677995

In [16]:
pagadas['FACTURAS_PAGADAS'].sum()

65348

In [ ]:
#QUERY QUE DA DE LOS QUE MIGRARON EL VALOR TOTAL PAGADO ANTES DE MIGRAR en recargas
query=f'''
SELECT A.SUBSCRIBER_ID,
       SUM(A.VALOR_CARGA) AS VALOR_CARGA 
FROM DWH_BODEGA_WOM.FCT_RECARGAS A 
WHERE A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND   A.SERVICIO='Prepaid'
GROUP BY A.SUBSCRIBER_ID
'''

In [5]:
query=f'''
WITH BASE AS
(SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'),
BASE2 AS(
SELECT A.SUBSCRIBER_ID,
       ROUND(SUM(A.VALOR_CARGA),0) AS VALOR_CARGA 
FROM DWH_BODEGA_WOM.FCT_RECARGAS A 
WHERE A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND   A.SERVICIO='Prepaid'
AND   A.TIEMPO_RECARGA_DK>20220801
GROUP BY A.SUBSCRIBER_ID ),
BASE3 AS (
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.PERIODO=202210
AND   A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
)


SELECT A.SUBSCRIBER_ID,
       COALESCE(B.VALOR_CARGA,0) AS VALOR_CARGA,
       C.SEG_RFM_MES_ANT
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN BASE3 C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
'''
recargas= cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
query=f'''
WITH BASE AS
(SELECT SUBSCRIBER_ID
FROM TEMP_PAGOS_0),
BASE2 AS(
SELECT A.SUBSCRIBER_ID,
       ROUND(SUM(A.VALOR_CARGA),0) AS VALOR_CARGA 
FROM DWH_BODEGA_WOM.FCT_RECARGAS A 
WHERE A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM TEMP_PAGOS_0)
AND   A.SERVICIO='Prepaid'
GROUP BY A.SUBSCRIBER_ID ),
BASE3 AS (
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.PERIODO=202210
AND   A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM TEMP_PAGOS_0)
)


SELECT A.SUBSCRIBER_ID,
       COALESCE(B.VALOR_CARGA,0) AS VALOR_CARGA,
       C.SEG_RFM_MES_ANT
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN BASE3 C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
'''
recargas= cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
recargas

,SUBSCRIBER_ID,VALOR_CARGA,SEG_RFM_MES_ANT
0,1112637954,1000,4.3 Sin Ingresos - On demand
1,1112640906,4000,4.1 Sin Ingresos - Inbound
2,1112662757,2000,4.2 Sin Ingresos - Whatsapp
3,1115995159,52000,3.3 Esporadico - Low
4,1112664556,81000,4.3 Sin Ingresos - On demand
...,...,...,...
1219,2245458539,5000,4.3 Sin Ingresos - On demand
1220,2248149224,5000,4.3 Sin Ingresos - On demand
1221,2248414622,18000,3.3 Esporadico - Low
1222,2264461628,3000,3.3 Esporadico - Low


In [15]:
recargas.to_csv('recargas0.csv')

In [6]:
filtro=recargas['VALOR_CARGA']==0
resultado=recargas.loc[filtro]


In [7]:
resultado

,SUBSCRIBER_ID,VALOR_CARGA,SEG_RFM_MES_ANT
4,1316576434,0,1.1 Gross 0 - 5
17,1302091359,0,1.2 Gross 6 - 15
21,1315040319,0,1.2 Gross 6 - 15
58,1196617260,0,4.3 Sin Ingresos - On demand
68,1161505987,0,4.3 Sin Ingresos - On demand
...,...,...,...
15052,1265429559,0,4.2 Sin Ingresos - Whatsapp
15083,1242710135,0,3.3 Esporadico - Low
15086,1156852377,0,4.3 Sin Ingresos - On demand
15094,1278175090,0,3.3 Esporadico - Low


In [8]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [9]:
resultado.to_csv('total_recargas0.csv')

In [ ]:
df=recargas.groupby('SEG_RFM_MES_ANT').sum()

In [43]:
df

,SUBSCRIBER_ID,VALOR_CARGA
SEG_RFM_MES_ANT,,
1.1 Gross 0 - 5,1195512088631,21632753.97
1.2 Gross 6 - 15,777310599272,10835699.97
1.3 Gross 16 - 30,500491125535,5406000.00
1.4 Gross > 30,5211405405,34000.00
2.1 Recurrente - Heavy - 30x3,144257149801,4480800.00
2.2 Recurrente - Heavy,6705510366513,305702238.52
2.3 Recurrente - Medium,1453010537895,30052499.58
2.4 Recurrente - Low,449933286589,5213899.98
3.1 Esporadico - Heavy,2238189888321,65737144.80


In [41]:
df.to_csv('total_recargas.csv')

In [47]:
#QUERY PARA POSPAGO FACTURAS 3 MESES
query=f'''
WITH BASE AS
(SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'),
BASE2 AS(
SELECT A.SUBS_ID,
       -1*SUM(A.PAGO) AS PAGOS
       --COUNT(*) AS FACTURAS_PAGADAS
FROM CR_DL_PAGOS A
WHERE A.SUBS_ID IN (
SELECT SUBSCRIBER_ID FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
AND SALDO=0
AND TO_NUMBER(TO_CHAR(FECHA_PAGO,'YYYYMMDD'))<20230301
GROUP BY SUBS_ID ),
BASE3 AS (
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.PERIODO=202210
AND   A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND   A.SERVICIO='Prepaid'
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION')
)


SELECT A.SUBSCRIBER_ID,
       COALESCE(B.PAGOS,0) AS PAGOS,
       C.SEG_RFM_MES_ANT
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBS_ID
LEFT JOIN BASE3 C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
'''
pagos_post= cst.consulta(query)


c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
#QUERY PARA POSPAGO FACTURAS 3 MESES
query=f'''
WITH BASE AS
(SELECT SUBSCRIBER_ID
FROM TEMP_PAGOS_0),
BASE2 AS(
SELECT A.SUBS_ID,
       -1*SUM(A.PAGO) AS PAGOS
       --COUNT(*) AS FACTURAS_PAGADAS
FROM CR_DL_PAGOS A
WHERE A.SUBS_ID IN (
SELECT SUBSCRIBER_ID FROM TEMP_PAGOS_0)
AND SALDO=0
AND TO_NUMBER(TO_CHAR(FECHA_PAGO,'YYYYMMDD'))<20230301
GROUP BY SUBS_ID ),
BASE3 AS (
SELECT A.SUBSCRIBER_ID,
       A.SEG_RFM_MES_ANT
FROM mvaliente.tbl_anl_base_prepago A
WHERE A.PERIODO=202210
AND   A.SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM TEMP_PAGOS_0)
)


SELECT A.SUBSCRIBER_ID,
       COALESCE(B.PAGOS,0) AS PAGOS,
       C.SEG_RFM_MES_ANT
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBS_ID
LEFT JOIN BASE3 C ON A.SUBSCRIBER_ID=C.SUBSCRIBER_ID
'''
pagos_post= cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
df=pagos_post.groupby('SEG_RFM_MES_ANT').sum()

In [12]:
df

,SUBSCRIBER_ID,PAGOS
SEG_RFM_MES_ANT,,
1.1 Gross 0 - 5,44436097921,972000
1.2 Gross 6 - 15,53783409357,1526031
1.3 Gross 16 - 30,66993113399,2172741
2.1 Recurrente - Heavy - 30x3,31840959954,1549635
3.3 Esporadico - Low,482858161434,14040626
4.1 Sin Ingresos - Inbound,137612638366,4159882
4.2 Sin Ingresos - Whatsapp,219764594133,6388367
4.3 Sin Ingresos - On demand,405001949201,16244933
4.4 Sin Ingresos - Low Quality,54031818276,1936000


In [ ]:
#QUERRY QUE DA LOS QUE RECIBIERON LLAMADA

query=f'''
SELECT CAMPANIA,COUNT(*)
FROM CR_DL_GESTION_OCC A
WHERE A.NUMERO_CONTACTO IN (
SELECT A.MSISDN 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE A.PERIODO_PROCESO_CODIGO=202302
AND   A.MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
AND   A.SERVICIO='Prepaid')
AND   A.PERIODO_PROCESO_CODIGO >= 20230101
AND   A.FUE_TOMADA IN ('Si','Yes')
GROUP BY CAMPANIA
ORDER BY COUNT(*) DESC
'''

In [ ]:
#query para ver los canales 
query=f'''
SELECT CONSULTOR_CANAL_NOMBRE,COUNT(*)
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.SERVICIO='Postpaid'
AND A.MOVIMIENTO_TIPO_NOMBRE='GROSS ADDS MIGRACION'
AND A.PERIODO_PROCESO_CODIGO=202302
GROUP BY CONSULTOR_CANAL_NOMBRE
ORDER BY COUNT(*) DESC
'''

In [ ]:
#QUERY QUE TRAETODOS LOS QUE HICIERON AL MENOS 1 RECARGA EL ULTIMO Y COMPARA SU MIGRACION 
query=f'''
WITH BASE AS(
SELECT *
FROM DWH_BODEGA_WOM.FCT_RECARGAS
WHERE PERIODO_PROCESO_CODIGO IN (202306,202305)
AND SERVICIO='Prepaid'
AND SUBSCRIBER_ID IN (SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE PERIODO_PROCESO_CODIGO=202306
AND MOVIMIENTO_TIPO_NOMBRE='CHURN MIGRACION'
AND SERVICIO='Prepaid')
),
BASE2 AS (SELECT SUBSCRIBER_ID,FECHA_ALTA,REVENUE
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE PERIODO_PROCESO_CODIGO=202306
AND MOVIMIENTO_TIPO_NOMBRE='GROSS ADDS MIGRACION'
AND SERVICIO='Postpaid'),
BASE3 AS (SELECT SUBSCRIBER_ID,
       SEG_RFM_MES_ACT 
FROM MVALIENTE.TBL_SEG_RFM_PREPAGO_NEW
WHERE PERIODO=202306),
BASE4 AS (SELECT SUBS_ID,
       -1*PAGO AS FACTURA,
       CASE
            WHEN SALDO=0 THEN 1 ELSE 0 
       END AS PAGO
FROM CR_DL_PAGOS
WHERE SUBS_ID IN (
SELECT SUBSCRIBER_ID
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE PERIODO_PROCESO_CODIGO=202306
AND MOVIMIENTO_TIPO_NOMBRE='GROSS ADDS MIGRACION'
AND SERVICIO='Postpaid'
)AND NUMERO_FACTURA=1)

SELECT  RANGO_RECARGA,
        COUNT(*),
        SUM(RECARGA_ULTIMO_MES) AS TOTAL_PREPAGO,
        SUM(RECARGA_ULTIMO_MES)/COUNT(*) AS PROMEDIO_RECARGA,
        SUM(REVENUE)AS TOTAL_POSPAGO,
        SUM(RECARGA_ULTIMO_MES)-SUM(FACTURA) AS DIF,
        SUM(FACTURA),
        SUM(PAGO),
        ROUND(SUM(PAGO)/COUNT(*),2) AS GENTE_PAGA
        
FROM (
SELECT A.SUBSCRIBER_ID, 
       SUM(A.VALOR_CARGA)AS RECARGA_ULTIMO_MES,
       SUM(B.REVENUE)/COUNT(*) AS REVENUE,
       CASE 
            WHEN SUM(A.VALOR_CARGA)>=20000 THEN '>=20K'
            WHEN SUM(A.VALOR_CARGA)>=15000 AND SUM(A.VALOR_CARGA)<20000 THEN '15K-20K'
            WHEN SUM(A.VALOR_CARGA)>=10000 AND SUM(A.VALOR_CARGA)<15000 THEN '10K-15K'
            WHEN SUM(A.VALOR_CARGA)>=5000 AND SUM(A.VALOR_CARGA)<10000 THEN '5K-10K'
            WHEN SUM(A.VALOR_CARGA)<5000 THEN '0-5K'
       END AS RANGO_RECARGA,
       SUM(C.FACTURA)/COUNT(*) AS FACTURA,
       SUM(C.PAGO)/COUNT(*) AS PAGO 
FROM BASE A
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
LEFT JOIN BASE4 C ON A.SUBSCRIBER_ID=C.SUBS_ID
WHERE A.FECHA_RECARGA BETWEEN B.FECHA_ALTA-30 AND B.FECHA_ALTA
GROUP BY A.SUBSCRIBER_ID
) GROUP BY RANGO_RECARGA
'''